First, move within a simulation directory, which contains the directory 'output/' that contains snapshot files, and a file 'snapshot_times.txt' that lists all snapshot scale-factors/redshifts/times in the simulation and their corresponding file index number.

Ensure that gizmo_analysis and utilities directories are in your python path, then...

In [3]:
#import gizmo_analysis as gizmo
import gizmo  # I happen to save my 'gizmo_analysis' directory as 'gizmo', but do as you prefer.

import utilities as ut

In [ ]:
# you can access the files as named or use the aliases in __init__.py to keep it shorter 
# for example, these are the same:

gizmo.gizmo_io
gizmo.io

# read particle data

In [ ]:
# read star and dark matter particles at z = 0

part = gizmo.io.Read.read_snapshots(['star', 'dark'], 'redshift', 0)

In [ ]:
# alternately, read all particle species at z = 0

part = gizmo.io.Read.read_snapshots('all', 'redshift', 0)

In [ ]:
# each particle species is stored as its own dictionary
# 'star' = stars, 'gas' = gas, 'dark' = dark matter, 'dark.2', 'dark.3', etc = low-resolution dark matter

part.keys()

In [ ]:
# properties of star particles are stored as dictionary

part['star'].keys()

In [ ]:
# properties of dark matter particles are stored as dictionary

part['dark'].keys()

# particle properties

In [ ]:
# 3-D position of star particles (particle.number x dimension.number array) [Mpc comoving]

part['star']['position']

In [ ]:
# 3-D velocity of star particles (particle_number x dimension_number array) [km/s physical]

part['star']['velocity']

In [ ]:
# mass of star particles [M_sun]

part['star']['mass']

In [ ]:
# formation scale-factor of star particles

part['star']['form.scalefactor']

In [ ]:
# use .prop() to compute derived quantities,
# such as the age of the Universe [Gyr] when a star particle formed
# see gizmo.io.ParticleDictionaryClass for all options for derived quantities

part['star'].prop('form.time')

In [ ]:
# similarly, get the age of a star particle (the lookback time to when it formed) [Gyr]

part['star'].prop('age')

# metallicities

In [ ]:
# 'metallicities' are stored in the catalog as *mass fractions*
# one value for each element, in a particle_number x element_number array
# the first value is the mass fraction of all metals (everything not H, He)
# then He, C, N, O, etc

part['star']['massfraction']

In [ ]:
# get individual elements by their index

# total metal mass fraction (everything not H, He) is index 0
print(part['star']['massfraction'][:, 0])

# iron is index 10
print(part['star']['massfraction'][:, 10])

In [ ]:
# alternately use .prop() to compute derived quantities,
# including calling elements by their name
# see gizmo.io.ParticleDictionaryClass for all options for derived quantities

print(part['star'].prop('massfraction.metals'))
print(part['star'].prop('massfraction.carbon'))
print(part['star'].prop('massfraction.iron'))

In [ ]:
# also use .prop() to compute metallicity [Z/H]
# for example, iron metallicity [Fe/H] := 
#   log10((mass_iron / mass_hydrogen)_particle / (mass_iron / mass_hydrogen)_sun)

print(part['star'].prop('metallicity.total'))
print(part['star'].prop('metallicity.fe'))

In [ ]:
# also use .prop() to compute simple arithmetic combinations, such as [Mg/Fe]

part['star'].prop('metallicity.mg - metallicity.fe')

In [ ]:
# refer to ut.basic.constants for assumed solar values (and other constants)

ut.basic.constants.sun_composition

# additional information stored in sub-dictionaries

In [ ]:
# dictionary of useful information about the simulation

part.info

In [ ]:
# dictionary of information about this snapshot's index, scale-factor, redshift, time, lookback-time

part.snapshot

In [ ]:
# dictionary of arrays about *all* snapshots that were saved for the simulation

print(part.Snapshot.keys())
print(part.Snapshot['redshift'][:10])

In [ ]:
# position [kpc comoving] and velocity [km/s physical] of the center of the host galaxy
# this was computed during read in, using ut.particle.get_center_position() and ut.particle.get_center_velocity()
# functions in gizmo.analysis use these values in computing profiles

print(part.center_position)
print(part.center_velocity)

See gizmo.analysis for examples of high-level analysis, including plotting these data.

See ut.particle for mid-level analysis functions that may be useful.

See other modules within utilities for low-level functions that may be useful.

In [ ]:
# for example, this computes the distance of star particles along the major and minor axis of the stellar disk
# here, radius_major is the radius (positive definite) [kpc physical] along the major axis wrt to the center position in the catalog
# radius_minor is the (signed) vertical height [kpc physical] wrt the disk
# axis_distance_max defines the maximum 3-D radius to use in determining the principal axes of the galaxy
# for a MW-mass galaxy, use ~10-20 [kpc]

radius_major, radius_minor = ut.particle.get_distances_along_principal_axes(part, 'star', '2d', axis_distance_max=15)

# particle tracking

For simulations, I generated pre-compiled HDF5 files to help with particle tracking. These are stored in the directory 'track/' (if present). The code that generates and reads these files is in gizmo_track.py.

star\_indices\_\*.hdf5 files store, for each star particles at z = 0, a pointer to where it was in the catalog at each previous snapshot (replace * with snapshot index). This makes it easy to quickly get the properties of a given star particle at any previous snapshot. These pointers are stored in an HDF5 file, one for each previous snapshot.

In [4]:
# first, read catalog of star particles at z = 0

part_at_z0 = gizmo.io.Read.read_snapshots(['star'], 'redshift', 0)


# in utilities.simulation.Snapshot():
  read snapshot_times.txt
  input redshift = 0.000 -> index = 600
  reading snapshot index = 600, redshift = 0.000

# in gizmo.gizmo_io.Read():
* reading header from: output/snapshot_600.hdf5
  species = star      (id = 4): 2770491 particles

* reading particles from: output/snapshot_600.hdf5

* checking sanity of particle properties

* assigning center of galaxy/halo:
  position = (41802.717, 44167.633, 46274.312) [kpc comoving]
  velocity = (-54.6, 72.4, 98.2) [km / s]



In [6]:
# say that you want to find out what they were doing at z = 1
# then, read in catalog of star particles at z = 1 (snapshot 277)

part_at_z1 = gizmo.io.Read.read_snapshots(['star'], 'redshift', 1)


# in utilities.simulation.Snapshot():
  read snapshot_times.txt
  input redshift = 1.000 -> index = 277
  reading snapshot index = 277, redshift = 1.000

# in gizmo.gizmo_io.Read():
* reading header from: output/snapshot_277.hdf5
  species = star      (id = 4): 1173962 particles

* reading particles from: output/snapshot_277.hdf5

* checking sanity of particle properties

* assigning center of galaxy/halo:
  position = (42418.086, 43285.416, 45265.248) [kpc comoving]
  velocity = (-62.6, 91.1, 84.6) [km / s]



In [15]:
# use the function within gizmo_track.py to read star index pointers associated with the catalog z = 1

gizmo.track.IndexPointer.io_index_pointer(part_at_z1)


# in utilities.basic.io.file_hdf5():
  reading file: star_indices_277.hdf5
  read indices: dtype = int32, shape = (2770491,)


In [16]:
# pointers are stored via numpy array appended to particle dictionary at the relevant snapshot
# a negative value means that the star formed after this snapshot (so it does not exist at this snapshot)

part_at_z1.index_pointers

array([  780991, -2770492, -2770492, ..., -2770492, -2770492,   154830], dtype=int32)

In [17]:
# so, say that you have a list of the indices of star particles of interest at z = 0

indices_at_z0 = np.array([3, 5, 8, 13])

In [19]:
# their positions at z = 0

part_at_z1['star']['position'][indices_at_z0]    

array([[ 42418.32588483,  43285.4441515 ,  45265.2785541 ],
       [ 42418.32200238,  43285.46701642,  45265.27616202],
       [ 42418.3182859 ,  43285.46377251,  45265.33799612],
       [ 42418.33740915,  43285.46897576,  45265.37083788]])

In [20]:
# now get their indices in the catalog at z = 1

indices_at_z1 = part_at_z1.index_pointers[indices_at_z0]
print(indices_at_z1)

[324959 348477 713157 654770]


In [21]:
# now easily get any property of interest at z = 1, for example, positions

part_at_z1['star']['position'][indices_at_z1]    

array([[ 42396.20211122,  43259.65199944,  45272.08530312],
       [ 42424.86951728,  43274.55010076,  45278.35258422],
       [ 42421.05527489,  43280.56907557,  45263.83724823],
       [ 42415.95423581,  43279.64531825,  45266.18323513]])

Another useful file: star\_form\_host\_distance\_600.hdf5 stores, for each star particle at z = 0, its distance at the first snapshot after it formed (formation distance) wrt to the main host galaxy, in kpc physical. This file includes both total (scalar) distance and full 3-D vector distance.

In [22]:
# use the function within gizmo_track.py to read this file and assign values directly to the catalog at z = 0

gizmo.track.HostDistance.io_form_host_distance(part_at_z0)


# in utilities.basic.io.file_hdf5():
  reading file: star_form_host_distance_600.hdf5
  read form.host.distance: dtype = float32, shape = (2770491,)
  read form.host.distance.3d: dtype = float32, shape = (2770491, 3)
  read id: dtype = uint32, shape = (2770491,)


In [23]:
# total (scalar) distance at formation, wrt host galaxy [kpc physical]

part_at_z0['star']['form.host.distance']

array([   10.38123703,   850.24914551,  1582.97070312, ...,  1032.54345703,
         882.52648926,   283.66540527], dtype=float32)

In [24]:
# 3-D distance vector at formation, wrt host galaxy [kpc physical]
# these distances are aligned with the principal axes of the host galaxy,
# principal axes defined according to all star particles within the host galaxy, independently at each snapshot
# distance along dimension 0 is aligned with the major axis
# distance along dimension 1 is algined with the intermediate axis
# distance along dimension 2 is aligned with the minor (Z) axis

part_at_z0['star']['form.host.distance.3d']

array([[    2.54848027,    -8.07790375,    -6.0019002 ],
       [  141.68244934,  -332.87545776,   769.44366455],
       [ 1454.2208252 ,   174.99990845,  -600.34399414],
       ..., 
       [  343.00650024,  -866.27142334,   445.0463562 ],
       [  -28.1642437 ,  -534.58935547,  -701.62237549],
       [  196.24163818,   198.28691101,    51.35728455]], dtype=float32)

In [25]:
# for a disk galaxy, you may want to combine the major and intermediate axes into an absolute 'radius'
# this function is useful: returns scalar (absolute) radius and (signed) vertical height within the disk

radius_in_disk, heigh_in_disk = ut.coordinate.get_distances_2d(part_at_z0['star']['form.host.distance.3d'])
print(radius_in_disk, heigh_in_disk)

[    8.47037601   361.77337646  1464.71264648 ...,   931.70788574
   535.33074951   278.97756958] [  -6.0019002   769.44366455 -600.34399414 ...,  445.0463562  -701.62237549
   51.35728455]


In [30]:
# these value look more reasonable is restrict to star particles that formed within the host galaxy

pis = ut.array.get_indices(part_at_z0['star']['form.host.distance'], [0, 8])  # 0 - 8 kpc physical
radius_in_disk, heigh_in_disk = ut.coordinate.get_distances_2d(part_at_z0['star']['form.host.distance.3d'][pis])
print(radius_in_disk, heigh_in_disk)

[ 5.8346076   2.00492787  0.27028641 ...,  0.38791442  1.86543489
  1.25457919] [ 0.85434961 -0.32502872  0.6857993  ..., -0.1979351   0.84220695
  0.10890873]
